# cli
> CLI commands
- order: 13

In [ ]:
#|default_exp cli

In [ ]:
#|export
from __future__ import annotations
import warnings

from nbdev.config import *
from nbdev.process import *
from nbdev.processors import *
from nbdev.doclinks import *
from nbdev.test import *
from nbdev.clean import *
from nbdev.quarto import nbdev_readme, refresh_quarto_yml
from nbdev.frontmatter import FrontmatterProc

from execnb.nbio import *
from fastcore.meta import *
from fastcore.utils import *
from fastcore.script import *
from fastcore.style import S
from fastcore.shutil import rmtree,move

from urllib.error import HTTPError
from contextlib import redirect_stdout
import os, tarfile, sys

In [ ]:
#|hide
import tempfile
from nbdev import show_doc
from fastcore.test import *

## Filter -

In [ ]:
#|export
@call_parse
def nbdev_filter(
    nb_txt:str=None,  # Notebook text (uses stdin if not provided)
    fname:str=None,  # Notebook to read (uses `nb_txt` if not provided)
    printit:bool_arg=True, # Print to stdout?
):
    "A notebook filter for Quarto"
    os.environ["IN_TEST"] = "1"
    try: filt = globals()[get_config().get('exporter', 'FilterDefaults')]()
    except FileNotFoundError: filt = FilterDefaults()
    if fname:        nb_txt = Path(fname).read_text()
    elif not nb_txt: nb_txt = sys.stdin.read()
    nb = dict2nb(loads(nb_txt))
    if printit:
        with open(os.devnull, 'w') as dn:
            with redirect_stdout(dn): filt(nb)
    else: filt(nb)
    res = nb2str(nb)
    del os.environ["IN_TEST"]
    if printit: print(res, flush=True)
    else: return res

In [ ]:
#|hide
# res = nbdev_filter(fname=get_config().nbs_path/'API'/'merge.ipynb', printit=False)
# print(res)

## New -

In [ ]:
#|export
def extract_tgz(url, dest='.'):
    from fastcore.net import urlopen
    with urlopen(url) as u: tarfile.open(mode='r:gz', fileobj=u).extractall(dest)

In [ ]:
#|export
def _render_nb(fn, cfg):
    "Render templated values like `{{lib_name}}` in notebook at `fn` from `cfg`"
    txt = fn.read_text()
    txt = txt.replace('from your_lib.core', f'from {cfg.lib_path}.core') # for compatibility with old templates
    for k,v in cfg.d.items(): txt = txt.replace('{{'+k+'}}', v)
    fn.write_text(txt)

In [ ]:
#|export
def _update_repo_meta(cfg):
    "Enable gh pages and update the homepage and description in your GitHub repo."
    token=os.getenv('GITHUB_TOKEN')
    if token: 
        from ghapi.core import GhApi
        api = GhApi(owner=cfg.user, repo=cfg.repo, token=token)
        try: api.repos.update(homepage=f'{cfg.doc_host}{cfg.doc_baseurl}', description=cfg.description)
        except HTTPError:print(f"Could not update the description & URL on the repo: {cfg.user}/{cfg.repo} using $GITHUB_TOKEN.\n"
                  "Use a token with the correction permissions or perform these steps manually.")

In [ ]:
#|export
@call_parse
@delegates(nbdev_create_config)
def nbdev_new(**kwargs):
    "Create an nbdev project."
    from ghapi.core import GhApi
    nbdev_create_config.__wrapped__(**kwargs)
    cfg = get_config()
    _update_repo_meta(cfg)

    path = Path()
    with warnings.catch_warnings():
        warnings.simplefilter('ignore', UserWarning)
        tag = GhApi(gh_host='https://api.github.com', authenticate=False).repos.get_latest_release('fastai', 'nbdev-template').tag_name
    url = f"https://github.com/fastai/nbdev-template/archive/{tag}.tar.gz"
    extract_tgz(url)
    tmpl_path = path/f'nbdev-template-{tag}'

    cfg.nbs_path.mkdir(exist_ok=True)
    nbexists = bool(first(cfg.nbs_path.glob('*.ipynb')))
    _nbs_path_sufs = ('.ipynb','.css')
    for o in tmpl_path.ls():
        p = cfg.nbs_path if o.suffix in _nbs_path_sufs else path
        if o.name == '_quarto.yml': continue
        if o.name == 'index.ipynb': _render_nb(o, cfg)
        if o.name == '00_core.ipynb' and not nbexists: move(o, p)
        elif not (path/o.name).exists(): move(o, p)
    rmtree(tmpl_path)

    refresh_quarto_yml()

    nbdev_export.__wrapped__()
    nbdev_readme.__wrapped__()

It works by copying contents from the latest tagged [`nbdev-template`](https://github.com/fastai/nbdev-template) release, and rendering termplate variables using the created settings.ini file. Settings can be passed via command line args; missing settings are inferred from the current git/GitHub repo, otherwise prompted for.

In [ ]:
#|hide
# get_config.cache_clear()
# with tempfile.TemporaryDirectory() as d, working_directory(d):
#     nbdev_new.__wrapped__(repo='my-project',branch='main',user='fastai',author='fastai',author_email='info@fast.ai',description='A test project')

## Help

In [ ]:
#|export
@call_parse
def chelp():
    "Show help for all console scripts"
    from fastcore.xtras import console_help
    console_help('nbdev')

In [ ]:
#|exec_doc
chelp()

nbdev_bump_version              Increment version in settings.ini by one
nbdev_changelog                 Create a CHANGELOG.md file from closed and labeled GitHub issues
nbdev_clean                     Clean all notebooks in `fname` to avoid merge conflicts
nbdev_conda                     Create a `meta.yaml` file ready to be built into a package, and optionally build and upload it
nbdev_create_config             Create a config file.
nbdev_docs                      Create Quarto docs and README.md
nbdev_export                    Export notebooks in `path` to Python modules
nbdev_filter                    A notebook filter for Quarto
nbdev_fix                       Create working notebook from conflicted notebook `nbname`
nbdev_help                      Show help for all console scripts
nbdev_install                   Install Quarto and the current library
nbdev_install_hooks             Install Jupyter and git hooks to automatically clean, trust, and fix merge conflicts in notebooks
n

## Export -

In [ ]:
#|hide
import nbdev; nbdev.nbdev_export()